<img src="images/quantum_algorithms_tutorials.png" alt="drawing" width="100" align="left"/>

<h2 align="center">
Hamiltonian simulation
</h2>

<h3 align="left">
Introduction
</h3>

Hamiltonian simulation is one of the main subroutines of many quantum algorithms. Moreover, any quantum circuit can be described as Hamiltonian simulation. This is becouse every quantum circuit (without noise) can be described by unitary transformation and for any unitary transformation $U$ we can find Hamiltonian $H$, such that $U = e^{-iHt}$, where $t$ is a real number. In the circuit model quantum computation the problem of Hamiltonian simulation can be described this way: for given $H$ Hamiltonian operator construct $U = e^{iHt}$ circuit. This problem comes from Schrödinger’s equation: the dynamical behavior of many simple quantum systems is governed by Schrödinger’s equation [[1](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)]:

$$i \hbar \frac{d}{dt} |\psi \rangle = H |\psi \rangle$$

where $|\psi \rangle$ is the quantum state, $\hbar$ is Plank's constant, $H$ is the Hamiltonian operator. When $H$ is not dependent on doesn't have dibendence on $t$ time, the solution of the Schrödinger’s equation will be [[1](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)]:

$$|\psi(t) \rangle = e^{-iHt} |\psi(0) \rangle$$

Hamiltonian operators are Hermitian operators, but $e^{-iHt}$ is an Unitary operator, that can be implemented by quantum gates. This is why Hamiltonian simulation is very important sobroutine for quantum system simulations (e.g. for quantum chemistry). It is the one of the main subroutines in the quantum phase estimation algoithm ([QPE](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/quantum_phase_estimation.ipynb) and [IQPE](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_quantum_phase_estimation.ipynb)), which is frequently used as a subroutine in other quantum algorithms, such as Shor's algorithm and the quantum algorithm for linear systems of equations (HHL algorithm) [[2](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)]. Also, we should mention that Hamiltonian simulation is used in the QAOA algorithm. So, it is hard to overestimate the importance of Hamiltonian simulation in the quantum computation.

<h3 align="left">
Constructing circuit for $e^{iHt}$
</h3>

Any Hermitian (Hamiltonian) matrix $H$ can be decomposed by sum of Pauli products with real coefficients (see this thread in Quantum Computing SE [[3][1]]). An example for 3 qubit Hamiltonian:

$$H = 11 \sigma_z \otimes \sigma_z + 7 \sigma_z \otimes \sigma_x - 5\sigma_z \otimes \sigma_x \otimes \sigma_y$$

The final circuit for $e^{iHt}$ can be simulated via Trotter decomposition (chapter 4.1 from paper [[4][2]]):

$$e^{iHt} \approx \big(\prod_k e^{iP_k \Delta t }\big)^{t/ \Delta t}$$

where $P_k$ are the Pauli product terms. By decreasing $\Delta t$ we can decrease the error of the Trotter decomposition. If all $P_k$ Pauli products are commuting to each other, then we can take $\Delta t = t$. For this simulation, we need to know how to simulate individual Pauli products $e^{iP_k t}$. Let's start with the simplest one $e^{i \sigma_z \otimes \sigma_z \otimes ... \otimes \sigma_z t}$ (chapter 4.2 paper [[4][2]]). Here is the circuit for $e^{i \sigma_z \otimes \sigma_z t}$ from the paper [[4][2]]

<img src="images/exp_sigma_z.png" alt="drawing" width="200"/>

Here $R_z$'s argument is $-2t$ ($R_z(-2t) = e^{i\sigma_z t}$). Before showing why this is true let's introduce 2 formulas that we will us. 4.2 exercise from [[1][3]] textbook: Let $t$ be a real number and $A$ a matrix such that $A^2 = I$. Then

$$e^{iA t} = \cos(t) I + i \sin(t) A$$

For all Pauli terms, this $P_k^2 =I$ is true. So we can use this formula. For CNOT gate we have:

$$\mathrm{CNOT} = |0\rangle \langle 0 | \otimes I + |1\rangle \langle 1 | \otimes \sigma_x$$

By taking these formulas into account let's show that the circuit implements the $e^{i \sigma_z \otimes \sigma_z t}$ Pauli term:

\begin{align*}
e^{i \sigma_z \otimes \sigma_z t} = \cos(t) I + i \sin(t) \sigma_z \otimes \sigma_z 
\end{align*}

The circuit:

\begin{align*}
&\mathrm{CNOT} \left(I \otimes e^{i \sigma_z  t}\right) \mathrm{CNOT}= 
\big[|0\rangle \langle 0 | \otimes I + |1\rangle \langle 1 | \otimes \sigma_x \big] \\ 
&\big[ \cos(t) I\otimes I + i \sin(t) I \otimes \sigma_z \big]
\big[|0\rangle \langle 0 | \otimes I + |1\rangle \langle 1 | \otimes \sigma_x \big] = \\
&= \cos(t) I + i \sin(t) \sigma_z \otimes \sigma_z 
\end{align*}

So, the circuit implements what we want:

$$
e^{i \sigma_z \otimes \sigma_z t} =\mathrm{CNOT} \left(I \otimes e^{i \sigma_z  t}\right) \mathrm{CNOT}
$$

The circuit for the $e^{i \sigma_z \otimes \sigma_z \otimes \sigma_z t}$ Pauli term from the same paper [[4][2]]:

<img src="images/exp_sigma_zz.png" alt="drawing" width="400"/>

This also can be shown the same way. Moreover, this solution can be generalized for $e^{i \sigma_z \otimes \sigma_z \otimes ... \otimes \sigma_z t}$ Pauli term. 

Now, what if we have one $\sigma_x$ in the tensor product $P = P_1 \otimes \sigma_x^{(n)} \otimes P_2$, where $P_1$ and $P_2$ are also Pauli products, $n$ is the qubit number. Note that:

\begin{align*}
e^{iP_1 \otimes \sigma_x^{(n)} \otimes P_2t} &= \cos(t) I + i \sin(t) P_1 \otimes \sigma_x^{(n)} \otimes P_2 = \\
&= \cos(t) I + i \sin(t) P_1 \otimes \left(H \sigma_z^{(n)} H\right) \otimes P_2 = \\
&= H^{(n)} e^{iP_1 \otimes \sigma_z^{(n)} \otimes P_2t} H^{(n)}
\end{align*}

where $H^{(n)}$ is the Hadamard gate acting on $n$th qubit. The same can be shown for $\sigma_y$:

\begin{align*}
e^{iP_1 \otimes \sigma_y^{(n)} \otimes P_2t} = H_y^{(n)} e^{iP_1 \otimes \sigma_z^{(n)} \otimes P_2t} H_y^{(n)}
\end{align*}

where $H_y$ is a self-inverse gate, which has this nice property $\sigma_y = H_y \sigma_z H_y$: 

$$H_y = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -i \\
i & -1
\end{pmatrix}
$$

Now, we can simulate any Pauli term. For example, the circuit for $e^{i \sigma_y \otimes \sigma_y \otimes \sigma_x}$ will look like this:

$$e^{i \sigma_y \otimes \sigma_y \otimes \sigma_x} = \big[ H_y \otimes H_y \otimes H \big]  e^{i \sigma_z \otimes \sigma_z \otimes \sigma_z} \big[H_y \otimes  H_y \otimes H\big]  $$

<img src="images/exp_yyx.png" alt="drawing" width="400"/>

By applying appropriate rotations we can bring any Pauli term simulation problem to the simplest $e^{i \sigma_z \otimes \sigma_z \otimes ... \otimes \sigma_z t}$ Pauli term simulation problem, which solution we already know. With this approach, we can simulate any Pauli term, thus also any Hermitian operator.


  [1]: https://quantumcomputing.stackexchange.com/questions/8725/can-arbitrary-matrices-be-decomposed-using-the-pauli-basis
  [2]: https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441
  [3]: https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB
  [4]: https://quantumcomputing.stackexchange.com/a/11391/9459
  [5]: https://i.stack.imgur.com/RwJTw.png

<h3 align="left">
$I$ operators in the Pauli product terms
</h3>

Here we will try to show that we can ignore $I$ operators in the Pauli products when we try to construct circuits for them. For example, we will take $\sigma_z \otimes I \otimes \sigma_z$ operator and show that we can forget about the second qubit and simulate the circuit for $e^{i\sigma_z \otimes \sigma_z t}$ applied on the first and third qubits. We should proof that:

$$e^{i\sigma_z \otimes I \otimes \sigma_z t} = CNOT^{(1,3)}R_z^{(3)}(-2t)CNOT^{(1,3)}$$

For the left side we have:

$$e^{i\sigma_z \otimes I \otimes \sigma_z t} = \cos(t) I \otimes I \otimes I + \sin(t) \sigma_z \otimes I \otimes \sigma_z$$

For the right side:

\begin{align}
&CNOT^{(1,3)}R_z^{(3)}(-2t)CNOT^{(1,3)} = \big[| 0 \rangle \langle 0 | \otimes I \otimes I + | 1 \rangle \langle 1 | \otimes I \otimes X  \big] \\
&\big[ I \otimes I \otimes \big( \cos(t) I + i \sin(t) \sigma_z \big) \big]
\big[ | 0 \rangle \langle 0 | \otimes I \otimes I  + | 1 \rangle \langle 1 | \otimes I \otimes X \big]=
\\
&= \cos(t) I \otimes I \otimes I + \sin(t) \sigma_z \otimes I \otimes \sigma_z
\end{align}

So, they are equal to each other: we can forget about $I$ operator for constructing a circuit in this case. This solution can be generalized for $n$ $I$ operators between two $\sigma_z$ terms.

Also, let's proof that $I \otimes P \otimes I$ can be simulated by $e^{i I \otimes P \otimes I t} = I \otimes e^{i P t} \otimes I$, where $P$ is some Pauli product:

\begin{align}
&e^{i I \otimes P \otimes I t} = \cos(t) I \otimes I \otimes I + i\sin(t) I \otimes P \otimes I 
\\
&= I \otimes \big( \cos(t) I + i \sin(t) P \big) \otimes = I \otimes e^{i P t} \otimes I
\end{align}

In this manner it can be shown for general cases that we can ignore $I$ operators when we simulate Pauli terms in the quantum circuits.

<h3 align="left">
Controlled version of $e^{iHt}$
</h3>

Often in the algorithms (e.g. in HHL or PEA), we want to construct not the circuit for Hamiltonian simulation $e^{iHt}$, but the controlled version of it. First of all, note that if we have $ABC$ circuit, where $A$, $B$ and $C$ are operators, then the controlled version of that circuit will equal $cA$ $cB$ $cC$, where $c$ denotes control version of an operator. We already know that $e^{iHt}$ consists of $e^{iPt}$ terms, where $P$ is some Pauli product. So, for solving the problem we should find a way for constructing controlled versions of $e^{iPt}$ terms. Here is a general form for the circuit that implements any given $e^{iPt}$ and the controlled version of it (like was done similarly in this paper [[4][2]]):

<img src="images/exp_pauli_and_controled.png" alt="drawing" width="670"/>

where $O_i$ are gates from this set $\{I, H, H_y\}$, and they are chosen differently for each Pauli term. The right circuit in the picture implements the controlled version of the $e^{iPt}$, because, if control qubit is in the $|0\rangle$ state the $R_z$ gate will not work and the rest gates will cancel each other (they are self-inverse gates). 

When we are talking about the controlled version of the circuit we shouldn't forget about $e^{iIt}$ term (the global phase in the $e^{iHt}$). We should also construct a circuit that implements controlled version of $e^{iIt}$. Let's call it controlled-global phase ($CGP$) gate and try to implement it. The effect of $CGP$ for controlled qubit $\alpha|0_c\rangle + \beta|1_c\rangle$ acting on multi-qubit $| \psi \rangle$ state:

$$ CGP \left( \alpha|0_c\rangle + \beta|1_c\rangle \right) |\psi\rangle = \left( \alpha|0_c\rangle + e^{it}\beta|1_c\rangle \right) |\psi\rangle$$

where $t$ is the phase. This action can be done just by one Qiskit's [u1][3] phase gate acting on the control qubit [[1][5]]:

$$u1(t) = \begin{pmatrix} 1 & 0 \\ 0 & e^{it}\end{pmatrix}$$


  [1]: https://quantumcomputing.stackexchange.com/a/11373/9459
  [2]: https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441
  [3]: https://qiskit.org/documentation/_modules/qiskit/extensions/standard/u1.html
  [4]: https://quantumcomputing.stackexchange.com/q/9668/9459
  [5]: https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB

<h3 align="left">
The code
</h3>

Before going into Qiskit code, note that:

\begin{equation*}
H_y = u2(\frac{\pi}{2}, \frac{\pi}{2})
\qquad \qquad
H = u2(0, \pi)
\\
u2(\varphi, \lambda) = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & -e^{i\lambda} \\
e^{i\varphi} & e^{i(\varphi + \lambda)}
\end{pmatrix}
\end{equation*}

So, in the code we will use $u2(\frac{\pi}{2}, \frac{\pi}{2})$ gate for implementing $H_y$. For $H$ gate we can use either `.h()` or `.u2()` and for better readability we will use `.h()` in the code.

Firstly we should define what Hamiltonian we want to simulate. Let's do simulation for this Hamiltonian:

$$H = 2 \cdot \sigma_x \otimes \sigma_z \otimes \sigma_x + 5 \cdot \sigma_z \otimes \sigma_y \otimes I + 7 \cdot I \otimes \sigma_y \otimes \sigma_z$$

It can be shown that all three terms commute to each other, thus we can write $e^{iHt}$ in this form:

$$e^{iHt} = e^{i2 \cdot \sigma_x \otimes \sigma_z \otimes \sigma_x t} \; \cdot \; e^{i5 \cdot \sigma_z \otimes \sigma_y \otimes I t} \; \cdot \; e^{i7 \cdot I \otimes \sigma_y \otimes \sigma_z t}$$

This will help us to not worry about the Trotterization procidure and will let us to draw simpler circuit at the end. Here we start coding:

In [ ]:
from qiskit import *
from numpy import pi as PI

# put the Hamiltonian in the dictionary

hamiltonian = {"XZX": 2, "ZYI": 5, "IYZ": 7}

# define function for ZZ..Z exponantiation
def exp_all_z(circuit, quantum_register, qubit_list, 
              t=1, controlled=false, control_qubit=None):
    
    # the simple cases
    if len(qubit_list) == 0:
        if controlled:
            circuit.u1(t, control_qubit)
        return
    elif len(qubit_list) == 1:
        circuit.rz(-2*t, quantum_register[qubit_list[0]])
        return
    
    # the actual implementation of exp(iZZ..Z t)
    for i in range(len(qubit_list) - 1):
        circuit.cx(quantum_register[qubit_list[i]], 
                   quantum_register[qubit_list[i + 1]])
    if controlled:
        circuit.crz(-2*t, quantum_register[qubit_list[-1]])
    else:
        circuit.rz(-2*t, quantum_register[qubit_list[-1]])
    for i in reversed(range(len(qubit_list) - 1)):
        circuit.cx(quantum_register[qubit_list[i + 1]], 
                   quantum_register[qubit_list[i]])

In [ ]:
def exp_pauli(pauli, circuit, quantum_register, 
             t=1, controlled=false, control_qubit=None):
    
    # handling exceptions conected with control qubit
    if controlled and control_qubit == None:
        raise Exception("the control qubit is not given")
    elif control_qubit in quantum_register:
        raise Exception("the control qubit is in the target register")
    if len(pauli) != len(quantum_register):
        raise Exception("pauli string doesn't match to the quantum register")
    
    circuit_bracket = QuantumCircuit(quantum_register)
    
    for i in range(len(quantum_register)):
        
    

[1] [M.A. Nielsen, I.L. Chuang, Cambridge University Press New York, "Quantum Computation and Quantum Information: 10th Anniversary Edition
10th" (2011)](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)

[2] [Quantum phase estimation algorithm (wiki)](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)

[3] [Quantum computing SE question "Can arbitrary matrices be-decomposed using the pauli basis"](https://quantumcomputing.stackexchange.com/questions/8725/can-arbitrary-matrices-be-decomposed-using-the-pauli-basis)

[4]  [J.D. Whitfield,  J. Biamonte  and  A. Aspuru-Guzik,
Molecular Physics, "Simulation of electronic structure Hamiltonians using
quantum computers" (2011)](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)